In [22]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

Loading training and validation datasets 

In [23]:
#Converts.csv file to pandas dataframe format => prepares for Dataloader
class load_data(Dataset): 
    def __init__(self,csvfile): 
        self.data= pd.read_csv(csvfile)
        #Grabs all input features, in this case it is both the subtitle/word input_ids and the _ids: [all rows, all columns from 1 to 1024]
        self.x = self.data.iloc[:-1,1:1025].values
        #Grabs the labels. 1 for mischief, 0 for none : [all rows, just column 1025 for the labels]
        self.y = self.data.iloc[:-1,1025:1026].values

        #we convert the values into tensor.float datatypes
        self.x_train = torch.tensor(self.x, dtype = torch.float32)
        self.y_train = torch.tensor(self.y, dtype = torch.float32)

    def __len__(self): 
        #get length for data set so we can use it for indexing
        return len(self.y_train)
    
    def __getitem__(self, idx):
        #grab (feature,label) pairs based on the index 
        return self.x_train[idx], self.y_train[idx]
        

In [24]:
#convert training and testing to csv
featureset1 = load_data('train.csv')
featureset2 = load_data('val.csv')

#Dataloader does final prep before we pass into the model(seperate into batchesm and shuffles up the data )
training_loader = DataLoader(featureset1,batch_size=50, shuffle=True)
validation_loader = DataLoader(featureset2, batch_size=50, shuffle =True)


Building the model

In [25]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    #three fully connected layers that result in a final 
    self.fc1 = nn.Linear(1024,512)
    self.fc2 = nn.Linear(512,512)
    self.fc3 = nn.Linear(512,1)
    
  
  def forward(self, input_ids):
    x = input_ids
    x = torch.sigmoid(self.fc1(x))
    x = torch.sigmoid(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
  


    return x

In [26]:
model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=.25)
#Binary Cross Entropy
loss_fn = nn.BCELoss()
#loss_fn = nn.CrossEntropyLoss()

In [27]:
def train(model,training_loader, validation_loader, optimizer,num_epochs): 
    for epoch in range(num_epochs):
        training_loss = 0
        for features,labels in training_loader: 
            #outputs the prediction
            outputs = model(features)
            optimizer.zero_grad()
            #BCE to generate loss(predictions, true label)
            loss = loss_fn(outputs, labels)
            training_loss += loss.item()
            loss.backward()
            optimizer.step()
            
        #validation loss
        validation_loss = 0
        for features,labels in validation_loader:
            val_outputs = model(features)
            val_loss = loss_fn(val_outputs, labels)
            optimizer.zero_grad()
            validation_loss += val_loss.item()
        
        training_loss /= len(training_loader)
        validation_loss /= len(validation_loader)

        print(f"Epoch {epoch}: Training loss {training_loss}, Validation loss {validation_loss}")
        
        

In [29]:
train(model, training_loader, validation_loader, optimizer, 500)

Epoch 0: Training loss 0.5799241803941273, Validation loss 0.5873525341351827
Epoch 1: Training loss 0.5703757305939993, Validation loss 0.5296493371327718
Epoch 2: Training loss 0.575436756724403, Validation loss 0.516759971777598
Epoch 3: Training loss 0.5761816530000596, Validation loss 0.5198110143343607
Epoch 4: Training loss 0.5640226432255336, Validation loss 0.5430507858594259
Epoch 5: Training loss 0.5764341042155311, Validation loss 0.5576398372650146
Epoch 6: Training loss 0.5781615333897727, Validation loss 0.543934166431427
Epoch 7: Training loss 0.5911565323670706, Validation loss 0.5629330972830454
Epoch 8: Training loss 0.5760028759638468, Validation loss 0.5134664277235667
Epoch 9: Training loss 0.5688708396185012, Validation loss 0.5404719114303589
Epoch 10: Training loss 0.5773288692746844, Validation loss 0.5322492520014445
Epoch 11: Training loss 0.5713449588843754, Validation loss 0.561183234055837
Epoch 12: Training loss 0.5806436708995274, Validation loss 0.6498